In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -q albumentations
!pip install -q segmentation-models-pytorch
!pip install -q exifread


In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import segmentation_models_pytorch as smp

import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm

from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau

from scipy.ndimage import center_of_mass

import folium

import exifread

import glob, cv2, torch


Set Dataset Paths

In [ ]:
# Set image and mask directories
train_image_dir = "/kaggle/input/deepcrack/DeepCrack-master/dataset/DeepCrack/train_img"
train_mask_dir  = "/kaggle/input/deepcrack/DeepCrack-master/dataset/DeepCrack/train_lab"

test_image_dir  = "/kaggle/input/deepcrack/DeepCrack-master/dataset/DeepCrack/test_img"
test_mask_dir   = "/kaggle/input/deepcrack/DeepCrack-master/dataset/DeepCrack/test_lab"



# Get all image names
all_images = sorted(os.listdir(train_image_dir))

# Train/val split
train_imgs, val_imgs = train_test_split(all_images, test_size=0.2, random_state=42)


In [ ]:


uav_image_dir = "/kaggle/input/uav-based-crack-detection-dataset/UAV-based crack dataset used for segmentation/image"
sample_img_path = os.path.join(uav_image_dir, sorted(os.listdir(uav_image_dir))[0])

with open(sample_img_path, 'rb') as f:
    tags = exifread.process_file(f)
    gps_tags = {tag: val for tag, val in tags.items() if "GPS" in tag}
    print("Extracted GPS tags:")
    print(gps_tags)


 Create Dataset Class

In [ ]:
# class DeepCrackDataset(Dataset):
#     def __init__(self, image_dir, mask_dir, transform=None):
#         self.image_dir = image_dir
#         self.mask_dir = mask_dir
#         self.transform = transform
#         self.images = sorted(os.listdir(image_dir))  # ensure consistent order

#     def __len__(self):
#         return len(self.images)

#     def __getitem__(self, idx):
#         img_name = self.images[idx]
#         mask_name = img_name.replace(".jpg", ".png")  # match .png mask to .jpg image

#         img_path = os.path.join(self.image_dir, img_name)
#         mask_path = os.path.join(self.mask_dir, mask_name)

#         # Load and preprocess
#         image = cv2.imread(img_path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#         mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
#         mask = mask / 255.0  # normalize binary mask
#         mask = torch.tensor(mask).unsqueeze(0).float()

#         # Normalize image to [0, 1]
#         image = torch.tensor(image).permute(2, 0, 1).float() / 255.0

#         return image, mask



class DeepCrackDataset(Dataset):
    def __init__(self, image_dir, mask_dir, image_list, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.images = image_list
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        mask_name = img_name.replace(".jpg", ".png")

        img_path = os.path.join(self.image_dir, img_name)
        mask_path = os.path.join(self.mask_dir, mask_name)

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE) / 255.0
        mask = torch.tensor(mask).unsqueeze(0).float()

        image = torch.tensor(image).permute(2, 0, 1).float() / 255.0

        return image, mask



Metric Functions

In [ ]:
def dice_coeff(pred, target, threshold=0.5):
    pred = torch.sigmoid(pred) > threshold
    target = target > 0.5
    intersection = (pred & target).float().sum((1, 2, 3))
    union = pred.float().sum((1, 2, 3)) + target.float().sum((1, 2, 3))
    dice = (2. * intersection + 1e-7) / (union + 1e-7)
    return dice.mean().item()

def iou_score(pred, target, threshold=0.5):
    pred = torch.sigmoid(pred) > threshold
    target = target > 0.5
    intersection = (pred & target).float().sum((1, 2, 3))
    union = (pred | target).float().sum((1, 2, 3))
    iou = (intersection + 1e-7) / (union + 1e-7)
    return iou.mean().item()


In [ ]:
# train_dataset = DeepCrackDataset(train_image_dir, train_mask_dir)
# train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

train_dataset = DeepCrackDataset(train_image_dir, train_mask_dir, train_imgs)
val_dataset   = DeepCrackDataset(train_image_dir, train_mask_dir, val_imgs)



In [ ]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=8, shuffle=False)


In [ ]:
images, masks = next(iter(train_loader))
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(images[0].permute(1, 2, 0))
plt.title("Image")
plt.subplot(1, 2, 2)
plt.imshow(masks[0][0], cmap='gray')
plt.title("Mask")
plt.show()


 Define U-Net Model

In [ ]:
# Define model with a pretrained encoder
model = smp.Unet(
    encoder_name="resnet34",        # Pretrained backbone
    encoder_weights="imagenet",     # Use weights trained on ImageNet
    in_channels=3,                  # RGB input
    classes=1,                      # Binary mask output
    activation=None                 # We'll apply sigmoid manually
).cuda()


 Set Up Loss, Optimizer, Metrics

In [ ]:


loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)



Training Loop

In [ ]:
num_epochs = 50


for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    total_dice = 0
    total_iou = 0

    for images, masks in tqdm(train_loader):
        images, masks = images.cuda(), masks.cuda()

        preds = model(images)
        loss = loss_fn(preds, masks)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Metrics
        total_loss += loss.item()
        total_dice += dice_coeff(preds, masks)
        total_iou += iou_score(preds, masks)

    avg_loss = total_loss / len(train_loader)
    avg_dice = total_dice / len(train_loader)
    avg_iou = total_iou / len(train_loader)

    print(f"Epoch {epoch+1}/{num_epochs} | Loss: {avg_loss:.4f} | Dice: {avg_dice:.4f} | IoU: {avg_iou:.4f}")

    scheduler.step(avg_loss)  # uses avg_loss to decide whether to reduce LR


In [ ]:
torch.save(model.state_dict(), "unet_deepcrack_best.pth")


In [ ]:
model.eval()

# Load one sample from the validation dataset
img, true_mask = val_dataset[0]
img_input = img.unsqueeze(0).cuda()

with torch.no_grad():
    pred_mask = model(img_input)
    pred_mask = torch.sigmoid(pred_mask).squeeze().cpu().numpy()


In [ ]:
plt.figure(figsize=(15, 4))

plt.subplot(1, 3, 1)
plt.imshow(img.permute(1, 2, 0))
plt.title("Validation Image")

plt.subplot(1, 3, 2)
plt.imshow(true_mask.squeeze(), cmap='gray')
plt.title("Ground Truth Mask")

plt.subplot(1, 3, 3)
plt.imshow(pred_mask > 0.5, cmap='gray')
plt.title("Predicted Mask")

plt.show()


[](http://)

In [ ]:
import os
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

def extract_gps_from_exif(image_path):
    try:
        image = Image.open(image_path)
        exif_data = image._getexif()
        if not exif_data:
            return None

        gps_info = {}
        for tag, value in exif_data.items():
            tag_name = TAGS.get(tag)
            if tag_name == "GPSInfo":
                for t in value:
                    sub_tag = GPSTAGS.get(t, t)
                    gps_info[sub_tag] = value[t]
                return gps_info
    except Exception as e:
        print(f"Error reading {image_path}: {e}")
    return None

# Change this path to point to your dataset
folder_path = "/kaggle/input/sut-cracking/SUT-Crack/SUT-Crack/1-Segmentation/Original Image"

for filename in os.listdir(folder_path):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        gps_data = extract_gps_from_exif(os.path.join(folder_path, filename))
        if gps_data:
            print(f"✅ {filename} GPS Data: {gps_data}")
        else:
            print(f"❌ {filename} has no GPS metadata.")


In [ ]:
import exifread

def extract_gps_from_image(image_path):
    with open(image_path, 'rb') as f:
        tags = exifread.process_file(f)
        gps_tags = {tag: tags[tag] for tag in tags if "GPS" in tag}
        return gps_tags


In [ ]:
import glob

image_dir = "/kaggle/input/sut-cracking/SUT-Crack/SUT-Crack/1-Segmentation/Original Image/"
gps_lookup = {}

for img_path in glob.glob(os.path.join(image_dir, "*.jpg")):
    gps = extract_gps_from_image(img_path)
    if gps:
        gps_lookup[os.path.basename(img_path)] = gps


In [ ]:
def dms_to_decimal(dms, ref):
    degrees = float(dms.values[0].num) / dms.values[0].den
    minutes = float(dms.values[1].num) / dms.values[1].den
    seconds = float(dms.values[2].num) / dms.values[2].den
    decimal = degrees + (minutes / 60.0) + (seconds / 3600.0)
    return -decimal if ref in ['S', 'W'] else decimal

def convert_to_degrees(value):
    """Helper function to convert EXIF DMS to decimal."""
    d, m, s = value
    return d + (m / 60.0) + (s / 3600.0)

def get_decimal_coords(gps_data):
    """Extracts decimal latitude and longitude from GPS EXIF."""
    try:
        lat = convert_to_degrees(gps_data['GPSLatitude'])
        if gps_data['GPSLatitudeRef'] != 'N':
            lat = -lat
        lon = convert_to_degrees(gps_data['GPSLongitude'])
        if gps_data['GPSLongitudeRef'] != 'E':
            lon = -lon
        return lat, lon
    except:
        return None, None


In [ ]:
import folium

folder_path = "/kaggle/input/sut-cracking/SUT-Crack/SUT-Crack/1-Segmentation/Original Image"
map_center = [36.35, 59.50]  # Use approximate center for initializing the map

m = folium.Map(location=map_center, zoom_start=13)

for filename in os.listdir(folder_path):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(folder_path, filename)
        gps_data = extract_gps_from_exif(image_path)
        if gps_data:
            lat, lon = get_decimal_coords(gps_data)
            if lat and lon:
                folium.Marker(
                    location=[lat, lon],
                    popup=f"Crack Detected: {filename}",
                    icon=folium.Icon(color="red", icon="wrench", prefix="fa")
                ).add_to(m)
            else:
                print(f"⚠️ Skipped (missing lat/lon): {filename}")
        else:
            print(f"❌ No GPS metadata: {filename}")

m.save("sut_crack_map.html")


In [ ]:
sut_img_dir = "/kaggle/input/sut-cracking/SUT-Crack/SUT-Crack/1-Segmentation/Original Image"

sut_images = sorted([f for f in os.listdir(sut_img_dir) if f.endswith(".jpg")])
sut_predictions = []

for img_name in sut_images[:100]:  # limit for now
    img_path = os.path.join(sut_img_dir, img_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_tensor = torch.tensor(img).permute(2, 0, 1).unsqueeze(0).float().cuda() / 255.0

    with torch.no_grad():
        pred = model(img_tensor)
        pred = torch.sigmoid(pred).squeeze().cpu().numpy()
    
    sut_predictions.append((img_name, pred))


In [ ]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

def get_decimal_from_dms(dms, ref):
    degrees, minutes, seconds = dms
    decimal = degrees + minutes / 60 + seconds / 3600
    if ref in ['S', 'W']:
        decimal = -decimal
    return decimal

def get_image_gps(image_path):
    image = Image.open(image_path)
    exif_data = image._getexif()
    if not exif_data:
        return None
    for tag, val in exif_data.items():
        if TAGS.get(tag) == "GPSInfo":
            gps_info = {GPSTAGS.get(t, t): val[t] for t in val}
            lat = get_decimal_from_dms(gps_info['GPSLatitude'], gps_info['GPSLatitudeRef'])
            lon = get_decimal_from_dms(gps_info['GPSLongitude'], gps_info['GPSLongitudeRef'])
            return (lat, lon)
    return None


In [ ]:
import folium
from scipy.ndimage import center_of_mass

m = folium.Map(location=[36.3, 59.5], zoom_start=13)

for img_name, pred_mask in sut_predictions:
    if pred_mask.max() < 0.5:  # skip if no crack detected
        continue

    # Calculate centroid of predicted crack
    pred_binary = (pred_mask > 0.5).astype(np.uint8)
    if pred_binary.sum() == 0:
        continue
    y, x = center_of_mass(pred_binary)

    gps = get_image_gps(os.path.join(sut_img_dir, img_name))
    if gps:
        folium.Marker(
            location=gps,
            popup=img_name,
            icon=folium.Icon(color='red', icon='road')
        ).add_to(m)

m.save("sut_crack_inference_map.html")


In [ ]:
import torch
import segmentation_models_pytorch as smp

# Define the model architecture
model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
    activation=None
).cuda()

# Load trained weights
model.load_state_dict(torch.load("/kaggle/working/unet_deepcrack_best.pth"))
model.eval()


In [ ]:
from PIL import Image
from torchvision import transforms

# Preprocessing: resize to match training size, normalize
preprocess = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])


In [ ]:
import os
import numpy as np
from PIL.ExifTags import TAGS, GPSTAGS
from PIL import Image as PILImage
from scipy.ndimage import center_of_mass
import folium

# Path to SUT-Crack UAV image folder
sut_image_dir = "/kaggle/input/sut-cracking/SUT-Crack/SUT-Crack/1-Segmentation/Original Image"

# Function to extract GPS from EXIF
def get_gps(img_path):
    img = PILImage.open(img_path)
    exif = img._getexif()
    if not exif: return None
    gps_info = {}
    for tag, value in exif.items():
        if TAGS.get(tag) == "GPSInfo":
            for t in value:
                gps_info[GPSTAGS.get(t, t)] = value[t]
            break
    if not gps_info: return None

    def to_deg(dms):
        d, m, s = dms
        return d + m / 60.0 + s / 3600.0

    lat = to_deg(gps_info['GPSLatitude'])
    if gps_info['GPSLatitudeRef'] != 'N': lat *= -1
    lon = to_deg(gps_info['GPSLongitude'])
    if gps_info['GPSLongitudeRef'] != 'E': lon *= -1
    return (lat, lon)

# Initialize map
crack_map = folium.Map(location=[36.2, 59.3], zoom_start=14)

# Loop through images
for fname in sorted(os.listdir(sut_image_dir))[:100]:  # sample limit
    if not fname.lower().endswith(".jpg"): continue
    img_path = os.path.join(sut_image_dir, fname)
    
    gps = get_gps(img_path)
    if not gps: continue
    
    # Load and preprocess
    image = PILImage.open(img_path).convert("RGB")
    image_resized = preprocess(image).unsqueeze(0).cuda()

    # Predict
    with torch.no_grad():
        pred = torch.sigmoid(model(image_resized)).squeeze().cpu().numpy()
        binary_mask = (pred > 0.5).astype(np.uint8)

    # Find center of mass of crack region
    if binary_mask.sum() == 0: continue  # no crack found
    cy, cx = center_of_mass(binary_mask)

    # Add marker to map
    folium.Marker(
        location=gps,
        popup=folium.Popup(f"<b>{fname}</b>", max_width=200),
        icon=folium.Icon(color="red", icon="wrench", prefix="fa")
    ).add_to(crack_map)

# Save map
crack_map.save("/kaggle/working/sut_crack_inference_map.html")


In [ ]:
import torch
import cv2
import numpy as np
from torchvision import transforms

def preprocess_image(image_path, target_size=(512, 512)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    resized = cv2.resize(image, target_size)
    tensor = transforms.ToTensor()(resized)
    return tensor.unsqueeze(0), image  # model input, original RGB


In [ ]:
import segmentation_models_pytorch as smp

def load_model():
    model = smp.Unet(encoder_name="resnet34", encoder_weights="imagenet", in_channels=3, classes=1)
    model.load_state_dict(torch.load("/kaggle/working/unet_deepcrack_best.pth", map_location=torch.device('cpu')))
    model.eval()
    return model


In [ ]:
def predict_crack_mask(model, image_tensor, threshold=0.5):
    with torch.no_grad():
        output = model(image_tensor)
        mask = torch.sigmoid(output).squeeze().numpy()
        binary_mask = (mask > threshold).astype(np.uint8)
    return binary_mask


In [ ]:
def visualize_result(original, mask):
    mask_resized = cv2.resize(mask, (original.shape[1], original.shape[0]))
    overlay = original.copy()
    overlay[mask_resized == 1] = [255, 0, 0]  # mark damage in red
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.title("Original")
    plt.imshow(original)
    plt.subplot(1, 2, 2)
    plt.title("Predicted Mask")
    plt.imshow(overlay)
    plt.show()


In [ ]:
img_path = "/kaggle/input/uav-based-crack-detection-dataset/UAV-based crack dataset used for segmentation/image/DJI0090.png"
tensor_input, original_img = preprocess_image(img_path)
model = load_model()
predicted_mask = predict_crack_mask(model, tensor_input)
visualize_result(original_img, predicted_mask)


In [ ]:
import exifread

def extract_gps_from_exif(image_path):
    with open(image_path, 'rb') as f:
        tags = exifread.process_file(f)

    def get_decimal_from_dms(dms, ref):
        degrees = dms.values[0].num / dms.values[0].den
        minutes = dms.values[1].num / dms.values[1].den
        seconds = dms.values[2].num / dms.values[2].den
        decimal = degrees + minutes/60 + seconds/3600
        if ref in ['S', 'W']:
            decimal *= -1
        return decimal

    try:
        lat = get_decimal_from_dms(tags['GPS GPSLatitude'], tags['GPS GPSLatitudeRef'])
        lon = get_decimal_from_dms(tags['GPS GPSLongitude'], tags['GPS GPSLongitudeRef'])
        return lat, lon
    except KeyError:
        print("GPS data not found.")
        return None, None


In [ ]:
from scipy.ndimage import center_of_mass

def get_crack_centroid(mask):
    if np.sum(mask) == 0:
        return None
    y, x = center_of_mass(mask)
    return int(x), int(y)


In [ ]:
import folium

def plot_crack_on_map(lat, lon, save_path="sut_crack_inference_map.html"):
    m = folium.Map(location=[lat, lon], zoom_start=20)
    folium.Marker(location=[lat, lon], popup="Detected Crack").add_to(m)
    m.save(save_path)
    print(f"Map saved to {save_path}")


In [ ]:
import base64
from io import BytesIO
from PIL import Image

def plot_crack_with_preview(lat, lon, original_img, mask, save_path="sut_crack_single_map.html"):
    # Resize and overlay the mask
    mask_resized = cv2.resize(mask, (original_img.shape[1], original_img.shape[0]))
    overlay = original_img.copy()
    overlay[mask_resized == 1] = [255, 0, 0]

    # Convert to PNG for popup
    img = Image.fromarray(overlay)
    buffer = BytesIO()
    img.save(buffer, format="PNG")
    encoded = base64.b64encode(buffer.getvalue()).decode()

    m = folium.Map(location=[lat, lon], zoom_start=20)
    html = f'<img src="data:image/png;base64,{encoded}" width="300">'
    popup = folium.Popup(html, max_width=300)
    folium.Marker([lat, lon], popup=popup, icon=folium.Icon(color="red")).add_to(m)
    m.save(save_path)
    print(f"✅ Map saved to: {save_path}")


In [ ]:
img_path = "/kaggle/input/sut-cracking/SUT-Crack/SUT-Crack/1-Segmentation/Original Image/100.jpg"

# Inference
tensor_input, original_img = preprocess_image(img_path)
model = load_model()
pred_mask = predict_crack_mask(model, tensor_input)

# ✅ Show crack detection result
visualize_result(original_img, pred_mask)

# GPS Tagging and Mapping
lat, lon = extract_gps_from_exif(img_path)
if lat and lon:
    plot_crack_on_map(lat, lon)


In [ ]:
def preprocess_image(image_path, target_size=(512, 512)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    resized = cv2.resize(image, target_size)
    tensor = transforms.ToTensor()(resized)
    return tensor.unsqueeze(0), image

def load_model():
    model = smp.Unet(encoder_name="resnet34", encoder_weights="imagenet", in_channels=3, classes=1)
    model.load_state_dict(torch.load("/kaggle/working/unet_deepcrack_best.pth", map_location=torch.device('cpu')))
    model.eval()
    return model

def predict_crack_mask(model, image_tensor, threshold=0.5):
    with torch.no_grad():
        output = model(image_tensor)
        mask = torch.sigmoid(output).squeeze().numpy()
        return (mask > threshold).astype(np.uint8)

def visualize_result(original, mask):
    mask_resized = cv2.resize(mask, (original.shape[1], original.shape[0]))
    overlay = original.copy()
    overlay[mask_resized == 1] = [255, 0, 0]
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.title("Original")
    plt.imshow(original)
    plt.subplot(1, 2, 2)
    plt.title("Predicted Mask")
    plt.imshow(overlay)
    plt.show()

In [ ]:
def extract_gps_from_exif(image_path):
    with open(image_path, 'rb') as f:
        tags = exifread.process_file(f)

    def get_decimal_from_dms(dms, ref):
        degrees = dms.values[0].num / dms.values[0].den
        minutes = dms.values[1].num / dms.values[1].den
        seconds = dms.values[2].num / dms.values[2].den
        decimal = degrees + minutes/60 + seconds/3600
        if ref in ['S', 'W']:
            decimal *= -1
        return decimal

    try:
        lat = get_decimal_from_dms(tags['GPS GPSLatitude'], tags['GPS GPSLatitudeRef'])
        lon = get_decimal_from_dms(tags['GPS GPSLongitude'], tags['GPS GPSLongitudeRef'])
        return lat, lon
    except KeyError:
        return None, None

In [ ]:
def plot_crack_with_preview(lat, lon, original_img, mask, save_path="sut_crack_single_map.html"):
    mask_resized = cv2.resize(mask, (original_img.shape[1], original_img.shape[0]))
    overlay = original_img.copy()
    overlay[mask_resized == 1] = [255, 0, 0]

    img = PILImage.fromarray(overlay)
    buffer = BytesIO()
    img.save(buffer, format="PNG")
    encoded = base64.b64encode(buffer.getvalue()).decode()

    m = folium.Map(location=[lat, lon], zoom_start=20)
    html = f'<img src="data:image/png;base64,{encoded}" width="300">'
    popup = folium.Popup(html, max_width=300)
    folium.Marker([lat, lon], popup=popup, icon=folium.Icon(color="red")).add_to(m)
    m.save(save_path)
    print(f"✅ Map saved to: {save_path}")

In [ ]:
img_path = "/kaggle/input/sut-cracking/SUT-Crack/SUT-Crack/1-Segmentation/Original Image/100.jpg"

tensor_input, original_img = preprocess_image(img_path)
model = load_model()
pred_mask = predict_crack_mask(model, tensor_input)
visualize_result(original_img, pred_mask)

lat, lon = extract_gps_from_exif(img_path)
if lat and lon:
    plot_crack_with_preview(lat, lon, original_img, pred_mask)